In [ ]:
!pip install requests aiohttp

Все запросы описаны в ml-model-server/server/main.py

В ml-model-server/server/models.py находится фабрика для создания моделей и абстрактный класс модели и три конкретные реализации(логистическая и линейная регрессии, knn). Туда же загружаются модели после обучения.

в ml-model-server/server/config.py загружается конфигурация сервера

в ml-model-server/server/schemas.py описаны входные параметры для всех запросов

In [78]:
import requests

BASE_URL = "http://127.0.0.1:8000"

Попытаемся загрузить модель, которой нет на сервере, ожидается, что бросится исключение с соответствующим сообщением.

In [80]:
payload = {"model_name": "test_model_name"}
response = requests.post(f"{BASE_URL}/load", json=payload)
data = response.json()
assert response.status_code == 400,f"Неожиданный статус: {response.status_code}"
assert data["detail"] == f"Модель с именем `{payload['model_name']}` не найдена на диске", "Неверное сообщение в ответе"
data["detail"]

'Модель с именем `test_model_name` не найдена на диске'

Теперь попытаемся обучить и сохранить какую-нибудь модель на сервер. 

In [81]:
payload = {
        "model_name": "test_model_name",
        "model_type": "LinearRegression",
        "config": {"params": {}},
        "X": [[0, 1], [2, 3]],
        "y": [3, 3]
}
response = requests.post(f"{BASE_URL}/fit", json=payload) 
data = response.json()
assert response.status_code == 200,f"Неожиданный статус: {response.status_code}"
assert data["message"] == f"Обучение модели `{payload['model_name']}` запущено.", "Неверное сообщение в ответе"
data["message"]

'Обучение модели `test_model_name` запущено.'

Попробуем снова запустить код загрузки модели

In [82]:
payload = {"model_name": "test_model_name"}
response = requests.post(f"{BASE_URL}/load", json=payload)
data = response.json()
assert response.status_code == 200,f"Неожиданный статус: {response.status_code}"
assert data["message"] == f"Модель {payload['model_name']} загружена.", "Неверное сообщение в ответе"
data["message"]

'Модель test_model_name загружена.'

Попробуем загрузить её ещё раз, чтобы убедиться, что модель не загружается повторно

In [83]:
payload = {"model_name": "test_model_name"}
response = requests.post(f"{BASE_URL}/load", json=payload)
data = response.json()
assert response.status_code == 400,f"Неожиданный статус: {response.status_code}"
assert data["detail"] == f"Модель с именем `{payload['model_name']}` уже загружена", "Неверное сообщение в ответе"
data["detail"]

'Модель с именем `test_model_name` уже загружена'

Теперь попробуем загрузить моделей более, чем разрешено в конфиге(в конфиге 2)

In [84]:
max_models = 3  
for i in range(max_models-1):
    payload = {
        "model_name": f"model_{i}",
        "model_type": "LinearRegression",
        "config": {"params": {}},
        "X": [[0, 1], [2, 3]],
        "y": [3, 3]
    }
    response = requests.post(f"{BASE_URL}/fit", json=payload) 
    assert response.status_code == 200,f"Неожиданный статус при обучении модели {i}: {response.status_code}"
    response = requests.post(f"{BASE_URL}/load", json=payload)
    assert response.status_code == 200, f"Ошибка загрузки модели {i}"

payload = {"model_name": "model_over_limit"}
response = requests.post(f"{BASE_URL}/load", json=payload)
data = response.json()
assert response.status_code == 400,f"Неожиданный статус: {response.status_code}"
assert data["detail"] == f"Достигнуто максимальное количество загруженных моделей", "Неверное сообщение в ответе"
data["detail"]

'Достигнуто максимальное количество загруженных моделей'

Попробуем выгрузить из памяти модель

In [85]:
payload = {"model_name": "test_model_name"}
response = requests.post(f"{BASE_URL}/unload", json=payload)
data = response.json()
assert response.status_code == 200,f"Неожиданный статус: {response.status_code}"
assert data["message"] == f"Модель {payload['model_name']} выгружена.", "Неверное сообщение в ответе"
data["message"]

'Модель test_model_name выгружена.'

Попробуем ещё раз, чтобы убедиться, что модели больше нет в списках загруженных

In [86]:
payload = {"model_name": "test_model_name"}
response = requests.post(f"{BASE_URL}/unload", json=payload)
data = response.json()
assert response.status_code == 404,f"Неожиданный статус: {response.status_code}"
assert data["detail"] == f"Модель с именем {payload['model_name']} не загружена", "Неверное сообщение в ответе"
data["detail"]

'Модель с именем test_model_name не загружена'

Далее удалим тестовую модель с диска.

In [87]:
payload = {"model_name": "test_model_name"}
response = requests.delete(f"{BASE_URL}/models/remove", params=payload)
data = response.json()
assert response.status_code == 200,f"Неожиданный статус: {response.status_code}"
assert data["message"] == f"Модель {payload['model_name']}.pkl удалена с диска.", "Неверное сообщение в ответе"
data["message"]

'Модель test_model_name.pkl удалена с диска.'

Попробуем ещё раз, чтобы убедиться, что более на диске этой модели нет

In [88]:
payload = {"model_name": "test_model_name"}
response = requests.delete(f"{BASE_URL}/models/remove", params=payload)
data = response.json()
assert response.status_code == 404,f"Неожиданный статус: {response.status_code}"
assert data["detail"] == f"Модель {payload['model_name']}.pkl не найдена на диске", "Неверное сообщение в ответе"
data["detail"]

'Модель test_model_name.pkl не найдена на диске'

Удалим оставшиеся модели

In [89]:
response = requests.delete(f"{BASE_URL}/models/remove_all")
data = response.json()
assert response.status_code == 200,f"Неожиданный статус: {response.status_code}"
assert data["message"] == "Все модели удалены с диска", "Неверное сообщение в ответе"
data["message"]

'Все модели удалены с диска'